## Import all packages

In [75]:
import pandas as pd
import numpy as np
import sys

sys.path.append('../../')
from utils import microarray_utils, clustering_utils, bokeh_ui_utils

In [76]:
full_filtered_df = pd.read_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv')
full_filtered_df = full_filtered_df.rename(columns={'Unnamed: 0': 'TTHERM_ID'})
num_genes = full_filtered_df.shape[0]
num_genes

19152

In [77]:
df_y_to_ttherm = pd.read_csv('../../new_raw_data/tgd2024/yf_ttherm_mapping_feb2024.csv')
df_y_to_ttherm.head(10)

,g2021,yf2024,ttherm2021
0,g13054,YF00003507,TTHERM_00655860
1,g20599,YF00028628,TTHERM_00927120
2,g15833,YF00037255,TTHERM_00567960
3,g6705,YF00004460,TTHERM_000503751
4,g2684,YF00037811,TTHERM_00804790
5,g11935,YF00026031,TTHERM_000418169
6,g3371,YF00019461,TTHERM_00716070
7,g7684,YF00036590,TTHERM_00446070
8,g13881,YF00018024,TTHERM_00188330
9,g17309,YF00011744,TTHERM_00264990


In [78]:
dict_y_to_ttherm = {yf: ttherm for yf, ttherm in zip(df_y_to_ttherm['yf2024'].values, df_y_to_ttherm['ttherm2021'].values)}
dict_y_to_ttherm

{'YF00003507': 'TTHERM_00655860',
 'YF00028628': 'TTHERM_00927120',
 'YF00037255': 'TTHERM_00567960',
 'YF00004460': 'TTHERM_000503751',
 'YF00037811': 'TTHERM_00804790',
 'YF00026031': 'TTHERM_000418169',
 'YF00019461': 'TTHERM_00716070',
 'YF00036590': 'TTHERM_00446070',
 'YF00018024': 'TTHERM_00188330',
 'YF00011744': 'TTHERM_00264990',
 'YF00000692': 'TTHERM_00301690',
 'YF00020018': 'TTHERM_00571650',
 'YF00036412': 'TTHERM_00476560',
 'YF00027771': 'TTHERM_001414123',
 'YF00019459': 'TTHERM_00716090',
 'YF00005353': 'TTHERM_00146080',
 'YF00007877': 'TTHERM_000672289',
 'YF00013833': 'TTHERM_00335730',
 'YF00034057': 'TTHERM_00011610',
 'YF00036153': 'TTHERM_00382170',
 'YF00014194': 'TTHERM_00853040',
 'YF00024625': 'TTHERM_00327330',
 'YF00009042': 'TTHERM_00427470',
 'YF00015223': 'TTHERM_00170160',
 'YF00000272': 'TTHERM_00155480',
 'YF00015664': 'TTHERM_00660400',
 'YF00005973': 'TTHERM_00522430',
 'YF00028690': 'TTHERM_00945200',
 'YF00001875': 'TTHERM_000492719',
 'YF00004

In [79]:
for idx, row in full_filtered_df.iterrows():
    yfid = (row['TTHERM_ID']).replace('.t1', '')
    if yfid in dict_y_to_ttherm:
        full_filtered_df.at[idx, 'TTHERM_ID'] = dict_y_to_ttherm[yfid]

In [80]:
full_filtered_df.sample(10)

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
14537,TTHERM_00327340,10.773618,10.989403,11.065294,10.799017,11.087099,11.112806,10.660190,10.684632,10.687728,...,11.593225,11.488482,11.516227,11.927727,11.289149,11.415202,11.060236,11.258905,11.129483,11.384966
10949,TTHERM_00085480,8.276416,8.446920,8.089007,8.642270,8.142404,8.354259,8.751928,8.248748,8.301301,...,8.127591,9.486536,9.110657,8.130595,9.228838,8.989960,8.067539,9.182331,9.082115,8.002445
5450,TTHERM_00378870,8.811268,6.983178,6.897162,10.636274,8.884337,7.920547,12.299199,6.508434,6.314173,...,8.998895,8.798531,8.111039,9.306170,8.902798,9.037694,9.435339,8.252974,8.589025,9.183250
1258,TTHERM_000492999,9.098951,10.535461,9.787314,8.822656,10.185934,9.857795,8.534965,9.947951,8.875961,...,9.510181,10.259588,10.103353,10.214483,8.729668,9.120835,9.844492,8.470747,9.064072,9.483005
15739,YF00026813.t1,5.375001,6.229024,5.743865,6.032439,6.262697,6.225184,7.274498,5.630629,6.042398,...,7.149624,7.099534,6.976397,7.299147,7.591851,6.803122,8.033052,7.619326,7.245369,7.725947
10674,TTHERM_00192090,13.934409,13.864013,13.927624,14.204454,13.910560,14.138800,14.294925,14.317336,14.329673,...,12.957440,12.158452,12.060108,12.765924,12.627031,12.512610,12.903182,12.736347,12.095628,12.515611
18323,YF00036450.t1,12.165865,12.537659,12.388528,12.267120,12.690706,12.513517,12.263154,12.470780,12.658532,...,11.616127,12.083610,12.198449,11.886846,11.726422,11.941293,11.679639,11.746561,11.965084,11.247487
16379,TTHERM_00361710,9.002196,8.904702,8.015434,9.781063,9.178928,8.591928,10.420731,9.213078,8.981560,...,10.972436,10.673925,10.487874,10.424635,10.464261,10.393257,10.417621,10.365936,10.389929,10.147218
10606,TTHERM_00190990,4.933992,5.875502,5.189554,5.901160,5.759631,5.732070,6.100000,6.031194,5.192368,...,6.257990,6.185887,6.519770,5.961305,6.249564,6.478827,6.022132,6.314880,6.338516,5.741463
8241,TTHERM_00924240,5.109260,5.502329,5.682322,5.162344,5.470206,5.716225,4.927862,5.662746,6.500066,...,6.742484,5.034625,5.557661,5.516072,5.344190,5.623403,5.849994,4.985603,5.950829,5.674366


In [81]:
full_filtered_df.to_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv', index=False)

In [82]:
full_filtered_df = pd.read_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv')

In [83]:
full_filtered_df.sample(10)

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
13908,TTHERM_00485820,10.420040,9.910008,10.382270,10.557936,10.057159,10.510063,10.307692,10.588427,11.012620,...,11.387868,11.554103,11.282410,11.301805,11.517548,11.738983,11.426558,11.696004,11.826972,11.657850
19147,TTHERM_01082890,6.058780,5.903592,4.855192,5.990497,6.495354,5.428483,6.951234,4.779803,5.303821,...,5.980283,5.385525,5.078892,6.115855,5.556961,5.104625,8.341311,6.424418,5.033267,7.561166
9435,YF00016462.t1,13.112652,13.019977,12.960564,13.201054,13.296789,13.154799,13.357736,13.601916,13.413564,...,14.052710,13.362579,13.345369,13.818146,13.989776,13.854104,14.102689,14.052456,13.962268,14.345314
15495,TTHERM_00069380,8.700394,10.074080,9.493859,8.300968,9.625134,8.880056,7.485480,9.379465,8.760094,...,7.829088,6.560449,7.155733,6.743266,7.114769,6.912429,6.529099,6.201791,6.671466,7.037209
2976,TTHERM_00723460,7.195055,7.679558,7.150518,7.929906,8.021572,8.167894,7.676737,7.334631,7.199529,...,7.357789,6.974493,7.234351,7.351167,7.563279,7.467686,7.954533,6.924600,7.389364,8.326993
15712,TTHERM_00276070,10.398654,10.651165,10.616982,11.231161,11.302039,11.179320,11.723129,12.136156,12.036526,...,13.453871,12.995542,13.194666,13.379984,12.942466,12.997845,13.290242,12.891131,12.995126,13.187043
4097,TTHERM_000920829,8.583990,9.783943,9.321804,6.480658,8.338714,8.281344,6.204086,8.079739,7.210351,...,6.569548,5.814487,5.636254,5.451830,5.811115,5.888381,5.939330,4.927344,5.486013,5.274639
11567,TTHERM_00312360,5.804226,6.401151,6.747221,5.733169,6.690719,6.643791,7.090192,7.368702,7.612040,...,7.489165,7.907693,7.750004,7.401454,8.307714,7.900177,7.560475,8.167172,8.148245,8.005196
4423,TTHERM_00670590,13.442646,13.789674,13.181668,13.144014,13.567756,13.164642,13.201896,13.490142,13.181240,...,12.987732,13.451874,12.988053,12.805333,13.050316,12.922535,13.185590,12.748641,13.017409,12.829956
9372,YF00016333.t1,11.225829,11.642924,11.432158,11.341411,11.519140,11.471608,12.675528,11.999710,12.005612,...,12.069563,12.407782,12.327568,12.299483,13.064803,13.177539,12.663367,12.671399,13.351436,13.375240


In [84]:
full_filtered_norm_df = microarray_utils.normalize_expression_per_gene(full_filtered_df)

In [85]:
full_filtered_df

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_00161860,12.713124,13.122156,13.190829,12.795304,13.118586,13.032352,12.574914,12.663691,13.034661,...,11.513557,10.471010,9.960254,11.023613,11.673585,11.485081,11.210493,12.384087,12.286663,12.064307
1,TTHERM_00161850,12.801491,13.816009,13.674157,12.359067,13.633952,13.615677,10.186907,12.650432,12.646102,...,9.835771,11.409396,11.766843,8.720915,11.643657,11.601037,7.363241,11.566631,11.881848,6.776089
2,TTHERM_00161840,9.510123,10.560219,10.515640,9.734283,9.777910,10.445625,9.018961,9.359294,9.454292,...,7.076463,5.645302,5.691725,6.281886,5.106207,5.198772,6.182419,5.034927,5.121846,5.328239
3,TTHERM_00161830,11.401541,11.656088,11.342039,10.903930,11.327460,11.285148,10.979501,11.088203,10.749591,...,12.149286,13.059034,12.736203,12.230047,12.797433,12.510993,12.813015,12.429498,12.418377,11.955661
4,TTHERM_00161790,8.708858,9.328160,9.402818,9.127735,9.211735,9.383828,8.572715,9.266176,9.691604,...,7.206194,7.721578,7.477487,7.557246,6.733335,6.619555,7.099290,6.627048,6.288987,6.970930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19147,TTHERM_01082890,6.058780,5.903592,4.855192,5.990497,6.495354,5.428483,6.951234,4.779803,5.303821,...,5.980283,5.385525,5.078892,6.115855,5.556961,5.104625,8.341311,6.424418,5.033267,7.561166
19148,YF00038376.t1,6.299673,7.739830,6.906786,6.771473,7.031683,6.861101,7.750896,7.765508,7.275954,...,5.627632,6.973269,7.015752,5.926255,7.289689,7.166236,6.480323,7.474055,8.106916,6.342419
19149,YF00038377.t1,5.037861,5.205597,5.149139,5.269116,5.112030,5.155678,5.772467,4.890137,4.834001,...,4.905057,4.913738,4.655880,5.063498,4.889071,4.747743,5.371899,4.619045,4.315400,5.431138
19150,TTHERM_000989489,6.110999,6.792428,7.022609,5.629642,6.858904,6.627021,5.896079,6.660849,6.722843,...,4.746455,6.404011,6.419964,5.323734,6.287401,6.594715,6.116355,5.349351,6.305338,5.415274


In [86]:
list_first_3_veg_phases = [
            #  'Ll', 
            #  'Lm', 
            #  'Lh', 
             'S0', 
             'S3', 
             'S6', 
            #  'S9', 
            #  # 'S12', 
            #  'S15', 
            #  'S24'
             ]

In [87]:
first_3_veg_phases_cols = [col for col in list(full_filtered_df.columns) if col.split('_')[0] in list_first_3_veg_phases]
first_3_veg_phases_cols

['S0_GSM285363',
 'S0_GSM285554',
 'S0_GSM285561',
 'S3_GSM285542',
 'S3_GSM285555',
 'S3_GSM285562',
 'S6_GSM285543',
 'S6_GSM285556',
 'S6_GSM285563']

In [88]:
((full_filtered_df[['TTHERM_ID']+first_3_veg_phases_cols])[(full_filtered_df[first_3_veg_phases_cols].sum(axis=1) < 30)]).shape

(0, 10)

In [89]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]


In [90]:
len(full_filtered_df['TTHERM_ID'].values)

19152

In [91]:
len(full_filtered_df['TTHERM_ID'].unique())

19152

In [92]:
[c for c in full_filtered_df.columns if c[0] != 'C']

['TTHERM_ID',
 'Ll_GSM283687',
 'Ll_GSM284355',
 'Ll_GSM284362',
 'Lm_GSM283690',
 'Lm_GSM284357',
 'Lm_GSM284363',
 'Lh_GSM283691',
 'Lh_GSM284360',
 'Lh_GSM284364',
 'S0_GSM285363',
 'S0_GSM285554',
 'S0_GSM285561',
 'S3_GSM285542',
 'S3_GSM285555',
 'S3_GSM285562',
 'S6_GSM285543',
 'S6_GSM285556',
 'S6_GSM285563',
 'S9_GSM285544',
 'S9_GSM285564',
 'S15_GSM285559',
 'S15_GSM285566',
 'S24_GSM285547',
 'S24_GSM285560']

In [93]:
full_filtered_norm_df.columns

Index(['TTHERM_ID', 'Ll_GSM283687', 'Ll_GSM284355', 'Ll_GSM284362',
       'Lm_GSM283690', 'Lm_GSM284357', 'Lm_GSM284363', 'Lh_GSM283691',
       'Lh_GSM284360', 'Lh_GSM284364', 'S0_GSM285363', 'S0_GSM285554',
       'S0_GSM285561', 'S3_GSM285542', 'S3_GSM285555', 'S3_GSM285562',
       'S6_GSM285543', 'S6_GSM285556', 'S6_GSM285563', 'S9_GSM285544',
       'S9_GSM285564', 'S15_GSM285559', 'S15_GSM285566', 'S24_GSM285547',
       'S24_GSM285560', 'C0_GSM285570', 'C0_GSM656230', 'C4_GSM285574',
       'C4_GSM285588', 'C4_GSM656234', 'C6_GSM285575', 'C6_GSM656232',
       'C8_GSM285576', 'C8_GSM285590', 'C10_GSM285578', 'C10_GSM285591',
       'C12_GSM285579', 'C12_GSM285592', 'C12_GSM656237', 'C14_GSM285580',
       'C14_GSM285593', 'C14_GSM656238', 'C16_GSM285582', 'C16_GSM285595',
       'C16_GSM656239', 'C18_GSM285583', 'C18_GSM285596', 'C18_GSM656240'],
      dtype='object')

In [94]:
full_filtered_df.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_00161860,12.713124,13.122156,13.190829,12.795304,13.118586,13.032352,12.574914,12.663691,13.034661,...,11.513557,10.471010,9.960254,11.023613,11.673585,11.485081,11.210493,12.384087,12.286663,12.064307
1,TTHERM_00161850,12.801491,13.816009,13.674157,12.359067,13.633952,13.615677,10.186907,12.650432,12.646102,...,9.835771,11.409396,11.766843,8.720915,11.643657,11.601037,7.363241,11.566631,11.881848,6.776089
2,TTHERM_00161840,9.510123,10.560219,10.515640,9.734283,9.777910,10.445625,9.018961,9.359294,9.454292,...,7.076463,5.645302,5.691725,6.281886,5.106207,5.198772,6.182419,5.034927,5.121846,5.328239
3,TTHERM_00161830,11.401541,11.656088,11.342039,10.903930,11.327460,11.285148,10.979501,11.088203,10.749591,...,12.149286,13.059034,12.736203,12.230047,12.797433,12.510993,12.813015,12.429498,12.418377,11.955661
4,TTHERM_00161790,8.708858,9.328160,9.402818,9.127735,9.211735,9.383828,8.572715,9.266176,9.691604,...,7.206194,7.721578,7.477487,7.557246,6.733335,6.619555,7.099290,6.627048,6.288987,6.970930


In [95]:
full_filtered_norm_df.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_00161860,0.877569,0.934730,0.944327,0.889053,0.934231,0.922180,0.858254,0.870660,0.922503,...,0.709931,0.564237,0.492860,0.641462,0.732295,0.705952,0.667578,0.831586,0.817971,0.786897
1,TTHERM_00161850,0.874870,1.000000,0.982504,0.820302,0.977545,0.975291,0.552390,0.856239,0.855705,...,0.509082,0.703171,0.747258,0.371577,0.732064,0.726808,0.204123,0.722564,0.761443,0.131704
2,TTHERM_00161840,0.817959,1.000000,0.992272,0.856818,0.864382,0.980134,0.732813,0.791812,0.808280,...,0.396068,0.147967,0.156014,0.258323,0.054511,0.070558,0.241079,0.042154,0.057222,0.093002
3,TTHERM_00161830,0.219155,0.304722,0.199153,0.051882,0.194253,0.180029,0.077285,0.113825,0.000000,...,0.470512,0.776326,0.667806,0.497660,0.688389,0.592101,0.693626,0.564706,0.560968,0.405424
4,TTHERM_00161790,0.754290,0.909130,0.927797,0.859019,0.880021,0.923049,0.720251,0.893633,1.000000,...,0.378588,0.507447,0.446418,0.466360,0.260363,0.231915,0.351860,0.233788,0.149265,0.319767


In [96]:
veg_filtered_norm_df = full_filtered_norm_df[[c for c in full_filtered_norm_df.columns if c[0] != 'C']]
sex_filtered_norm_df = full_filtered_norm_df[[c for c in full_filtered_norm_df.columns if not c[0] in ['L', 'S']]]

In [97]:
veg_mean = microarray_utils.get_geom_mean_expression(veg_filtered_norm_df)
veg_mean

,TTHERM_ID,Ll,Lm,Lh,S0,S3,S6,S9,S15,S24
0,TTHERM_00161860,0.918647,0.915059,0.883601,0.138049,0.567886,0.627147,0.836865,0.931852,0.977207
1,TTHERM_00161850,0.951664,0.922946,0.748697,0.330364,0.429642,0.302862,0.543650,0.345832,0.150115
2,TTHERM_00161840,0.934881,0.899618,0.777338,0.254968,0.042519,0.045233,0.088184,0.107507,0.184493
3,TTHERM_00161830,0.240177,0.140211,0.062631,0.423834,0.319548,0.429970,0.430041,0.486488,0.518095
4,TTHERM_00161790,0.862085,0.887176,0.867695,0.322869,0.196533,0.068478,0.286755,0.139586,0.223256
...,...,...,...,...,...,...,...,...,...,...
19147,TTHERM_01082890,0.222227,0.328934,0.226834,0.296225,0.244669,0.213504,0.474207,0.362997,0.228489
19148,YF00038376.t1,0.648828,0.629565,0.843982,0.154151,0.479962,0.514778,0.547126,0.230330,0.400878
19149,YF00038377.t1,0.295997,0.313492,0.299985,0.304867,0.178534,0.206746,0.099545,0.188599,0.063531
19150,TTHERM_000989489,0.570263,0.494155,0.512495,0.911010,0.597678,0.704188,0.418372,0.704125,0.644979


In [98]:
sex_mean = microarray_utils.get_geom_mean_expression(sex_filtered_norm_df)
sex_mean

,TTHERM_ID,C0,C4,C6,C8,C10,C12,C14,C16,C18
0,TTHERM_00161860,0.948293,0.599406,0.734006,0.656497,0.582819,0.544810,0.565011,0.701734,0.812055
1,TTHERM_00161850,0.645933,0.428819,0.241582,0.737482,0.858162,0.675161,0.598129,0.532826,0.508663
2,TTHERM_00161840,0.368001,0.583861,0.438695,0.198958,0.384473,0.325974,0.186392,0.118975,0.063913
3,TTHERM_00161830,0.670451,0.885191,0.957351,0.738172,0.725875,0.582262,0.643213,0.657373,0.508498
4,TTHERM_00161790,0.818356,0.516315,0.089105,0.114974,0.271510,0.414148,0.473190,0.280371,0.232308
...,...,...,...,...,...,...,...,...,...,...
19147,TTHERM_01082890,0.580201,0.480149,0.756565,0.673306,0.289735,0.220372,0.203723,0.385328,0.407548
19148,YF00038376.t1,0.480322,0.306840,0.389976,0.422348,0.039438,0.242149,0.546161,0.653791,0.742858
19149,YF00038377.t1,0.225291,0.711959,0.323804,0.154843,0.123972,0.180392,0.202705,0.246088,0.159865
19150,TTHERM_000989489,0.547239,0.341402,0.278941,0.322759,0.075894,0.099392,0.407648,0.489860,0.313279


Get the annnotations

In [99]:
complete_annot = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')

Add the TGD annotations for all the genes to the dataframe

In [100]:
complete_annot

,TTHERM_ID,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs,TGD2021_description
0,TTHERM_00444130,5911.EAS03030,0.0,1773.3,"2E3YF@1|root,2SAXT@2759|Eukaryota,3ZCCZ@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,RRM23 hypothetical protein
1,YF00036564.t1,5911.EAS03033,0.0,4305.4,"2CBWM@1|root,2RT0G@2759|Eukaryota,3ZFJW@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,TTHERM_00444170,5911.EAS03034,4.699999999999999e-226,738.6,"2BF0K@1|root,2S15V@2759|Eukaryota,3ZBPB@5878|C...",5878|Ciliophora,I,2-oxoacid dehydrogenases acyltransferase (cata...,-,-,...,-,-,-,-,-,-,-,-,2-oxoacid_dh,2-oxo acid dehydrogenase acyltransferase domai...
3,TTHERM_00444180,5911.EAS03035,0.0,1065.5,"28KDA@1|root,2QSU4@2759|Eukaryota,3ZCSX@5878|C...",5878|Ciliophora,S,Radial spoke head,-,-,...,-,-,-,-,-,-,-,-,Radial_spoke,radial spokehead protein
4,TTHERM_00444190,5911.EAS03036,8.300000000000001e-303,1000.1,"COG0814@1|root,KOG1304@2759|Eukaryota,3ZF1N@58...",5878|Ciliophora,E,Transmembrane amino acid transporter protein,-,-,...,"ko04138,map04138",-,-,-,"ko00000,ko00001,ko02000,ko04131",2.A.18.8,-,-,Aa_trans,transmembrane amino acid transporter protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25982,YF00007805.t1,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
25983,TTHERM_01260770,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,hypothetical protein
25984,YF00007807.t1,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
25985,YF00007808.t1,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [101]:
# checkout_df = complete_annot.loc[complete_annot['TTHERM_ID'].isin(checkout)]

In [102]:
# len(checkout)

In [103]:
# checkout_df.to_csv('../checkout_different_filtered_genes.csv', index=False)

In [104]:
# complete_annot.to_csv('/Users/eukarya/Downloads/tgn_complete_annot.csv', index=False)

## Round one of plotting and consensus 

First, cluster accoding to UMAP/Leiden in full-dimensional space, using Manhattan distance and setting the number of neighbors to 5

### LEIDEN CLUSTERING

In [105]:
phases = 'full'

In [106]:
# leiden_label_df_round_1, full_dists, sscore, modularity = clustering_utils.build_label_df(full_filtered_norm_df, phases, random_state=42, n_neighbors=3, leiden_type='modularity')
leiden_label_df_round_1, full_dists, sscore, modularity =  clustering_utils.build_label_df(full_filtered_norm_df, phases, random_state=42, n_neighbors=5, leiden_type='cpm', la_res_param=0.035)

In [107]:
leiden_label_df_round_1_arranged = bokeh_ui_utils.arrange_modules(full_filtered_norm_df, leiden_label_df_round_1, phases) 
leiden_label_df_round_1_arranged_sorted = leiden_label_df_round_1_arranged.sort_values(by=['label', 'TTHERM_ID'], ascending=False)

Index(['TTHERM_ID', 'Ll_GSM283687', 'Ll_GSM284355', 'Ll_GSM284362',
       'Lm_GSM283690', 'Lm_GSM284357', 'Lm_GSM284363', 'Lh_GSM283691',
       'Lh_GSM284360', 'Lh_GSM284364', 'S0_GSM285363', 'S0_GSM285554',
       'S0_GSM285561', 'S3_GSM285542', 'S3_GSM285555', 'S3_GSM285562',
       'S6_GSM285543', 'S6_GSM285556', 'S6_GSM285563', 'S9_GSM285544',
       'S9_GSM285564', 'S15_GSM285559', 'S15_GSM285566', 'S24_GSM285547',
       'S24_GSM285560', 'C0_GSM285570', 'C0_GSM656230', 'C4_GSM285574',
       'C4_GSM285588', 'C4_GSM656234', 'C6_GSM285575', 'C6_GSM656232',
       'C8_GSM285576', 'C8_GSM285590', 'C10_GSM285578', 'C10_GSM285591',
       'C12_GSM285579', 'C12_GSM285592', 'C12_GSM656237', 'C14_GSM285580',
       'C14_GSM285593', 'C14_GSM656238', 'C16_GSM285582', 'C16_GSM285595',
       'C16_GSM656239', 'C18_GSM285583', 'C18_GSM285596', 'C18_GSM656240'],
      dtype='object')
Index(['TTHERM_ID', 'label'], dtype='object')
19152


In [108]:
sscore, modularity

(0.0102865160962089, 0.5068106349359283)

In [109]:
leiden_label_df_round_1_arranged_sorted.head()

,TTHERM_ID,label
19031,YF00037926.t1,1243
17634,YF00034320.t1,1243
16254,YF00027759.t1,1243
12726,YF00021698.t1,1243
12543,YF00021259.t1,1243


In [110]:
leiden_label_df_round_1_arranged_sorted['label'].mode()

0    827
dtype: int64

In [111]:
leiden_label_df_round_1

,TTHERM_ID,label
0,TTHERM_00161860,961
1,TTHERM_00161850,981
2,TTHERM_00161840,292
3,TTHERM_00161830,432
4,TTHERM_00161790,498
...,...,...
19147,TTHERM_01082890,123
19148,YF00038376.t1,852
19149,YF00038377.t1,620
19150,TTHERM_000989489,638


In [112]:
max(leiden_label_df_round_1['label'].values)

1243

In [113]:
# leiden_label_df_round_1, veg_dists, veg_sscore, veg_modularity = clustering_utils.build_label_df(veg_filtered_norm_df, 'veg', lldf=leiden_label_df_round_1, random_state=42, n_neighbors=3)
# leiden_label_df_round_1, sex_dists, sex_sscore, sex_modularity = clustering_utils.build_label_df(sex_filtered_norm_df, 'sex', lldf=leiden_label_df_round_1, random_state=42, n_neighbors=3)

In [114]:
leiden_label_df_round_1

,TTHERM_ID,label
0,TTHERM_00161860,961
1,TTHERM_00161850,981
2,TTHERM_00161840,292
3,TTHERM_00161830,432
4,TTHERM_00161790,498
...,...,...
19147,TTHERM_01082890,123
19148,YF00038376.t1,852
19149,YF00038377.t1,620
19150,TTHERM_000989489,638


In [115]:
# max(leiden_label_df_round_1['leiden_label_veg'].values)

In [116]:
# max(leiden_label_df_round_1['leiden_label_sex'].values)

Check out how many clusters we have

In [117]:
# leiden_label_df_round_1.to_csv('../clustergrammer/leiden_label_df_round_1_nn2.csv', index=False)

In [118]:
max(leiden_label_df_round_1['label'].values)

1243

In [119]:
# max(leiden_label_df_round_1['leiden_label_veg'].values)

In [120]:
# max(leiden_label_df_round_1['leiden_label_sex'].values)

In [121]:
module_size_list = []
for l in leiden_label_df_round_1['label'].unique():
    df = leiden_label_df_round_1.loc[leiden_label_df_round_1['label'] == l]
    module_size_list.append(len(df))
    
np.mean(module_size_list)

15.395498392282958

In [122]:
np.median(module_size_list)

15.0

In [123]:
np.max(module_size_list)

57

In [124]:
labeled_annot = leiden_label_df_round_1.merge(complete_annot, on='TTHERM_ID')
labeled_annot.head()

,TTHERM_ID,label,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,...,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs,TGD2021_description
0,TTHERM_00161860,961,5911.EAR89668,0.0,3238.2,"COG1132@1|root,KOG0054@2759|Eukaryota,3ZD02@58...",5878|Ciliophora,Q,ABC transporter family protein,-,...,"ko01523,ko02010,ko04024,ko04976,map01523,map02...",-,-,-,"ko00000,ko00001,ko01000,ko02000",3.A.1.208.7,-,-,"ABC_membrane,ABC_tran",ABC transporter family protein
1,TTHERM_00161850,981,5911.EAR89667,8.499999999999999e-271,892.2,"COG0697@1|root,KOG3912@2759|Eukaryota,3ZBQJ@58...",2759|Eukaryota,EG,Nucleotide-sugar transporter,SLC35F6,...,"ko04714,map04714",-,-,-,"ko00000,ko00001,ko03000",-,-,-,"EamA,Nuc_sug_transp,SLC35F",triose-phosphate transporter family protein
2,TTHERM_00161840,292,5911.EAR89666,6.799999999999999e-113,369.6,"COG0633@1|root,KOG3309@2759|Eukaryota,3ZC6Z@58...",5878|Ciliophora,C,"2 iron, 2 sulfur cluster binding",-,...,-,-,-,-,"ko00000,ko03029",-,-,-,Fer2,2 iron 2 sulfur cluster-binding protein
3,TTHERM_00161830,432,5911.EAR89665,1.8e-238,784.9,"KOG3026@1|root,KOG3026@2759|Eukaryota,3ZBX8@58...",5878|Ciliophora,A,Tudor domain,-,...,"ko03040,map03040",-,-,-,"ko00000,ko00001,ko03041",-,-,-,-,nucleic acid-binding protein putative
4,TTHERM_00161790,498,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,hypothetical protein


In [125]:
leiden_label_df_round_1.loc[leiden_label_df_round_1['label']==0]

,TTHERM_ID,label
2120,YF00003346.t1,0
2124,YF00003359.t1,0
2254,YF00003639.t1,0
2300,YF00003768.t1,0
2301,YF00003769.t1,0
2447,YF00004038.t1,0
2458,YF00004063.t1,0
5005,YF00008259.t1,0
5027,YF00008319.t1,0
5058,YF00008373.t1,0


In [126]:
leiden_label_df_round_1.loc[leiden_label_df_round_1['TTHERM_ID']=='TTHERM_00532700']

,TTHERM_ID,label
13374,TTHERM_00532700,95


In [127]:
labeled_annot.loc[labeled_annot['label'] == 10][['TTHERM_ID', 'Description', 'TGD2021_description']]

,TTHERM_ID,Description,TGD2021_description
327,TTHERM_000309879,double-strand break repair via nonhomologous e...,PiggyBac-like protein
1828,YF00002723.t1,FR47-like protein,-
3364,TTHERM_00142430,"Galactose oxidase, central domain",kelch motif protein
4636,YF00007502.t1,-,-
4922,YF00008066.t1,-,-
4975,YF00008174.t1,-,-
5578,TTHERM_00561550,-,eukaryotic aspartyl protease family protein pu...
5850,YF00009814.t1,-,-
6036,TTHERM_00006290,transmembrane transport,MFS transporter
6253,YF00010598.t1,-,-


In [128]:
leiden_label_df_round_1_arranged_sorted.to_csv('./test_nn3_leiden_label_df_round_1.csv', index=False)

In [129]:
leiden_label_df_round_1

,TTHERM_ID,label
0,TTHERM_00161860,961
1,TTHERM_00161850,981
2,TTHERM_00161840,292
3,TTHERM_00161830,432
4,TTHERM_00161790,498
...,...,...
19147,TTHERM_01082890,123
19148,YF00038376.t1,852
19149,YF00038377.t1,620
19150,TTHERM_000989489,638


In [130]:
np.sum(np.array(module_size_list) >= 5)

1219

In [131]:
leiden_label_df_round_1.loc[leiden_label_df_round_1['TTHERM_ID'] == 'TTHERM_01386050']

,TTHERM_ID,label
12945,TTHERM_01386050,3


In [132]:
# full_pymde_data = full_filtered_norm_df[list(full_filtered_norm_df.columns)[1:]].values

# pymde.seed(42)
# torch.seed(42)

# full_embedding = pymde.preserve_neighbors(
#     full_pymde_data, 
#     verbose=True,
#     n_neighbors=5,
#     constraint=pymde.Standardized()
# ).embed()

# pymde_full_df = pd.DataFrame(np.array(full_embedding), columns=('x', 'y'))
# bokeh.plotting.output_file(filename='./plots/full_norm_pymde_leiden_round_1.2.html', mode='inline')

# p = plot_embedding(full_filtered_norm_df, pymde_full_df, complete_annot, leiden_label_df_round_1, 'full', palette32, n_neighbors=5, radius=0.005)
# bokeh.plotting.save(p)

For ordering the leiden clusters in the heatmap, try computing the cityblock distance between centroids and then ordering by smallest distance, starting with largest cluster.

In [133]:
test_centroid = pd.DataFrame({'a': np.array([1, 2, 3, 4]), 'b': np.array([0, 2, 0, 2])})
test_centroid

,a,b
0,1,0
1,2,2
2,3,0
3,4,2


In [134]:
test_centroid.mean(axis=0).values

array([2.5, 1. ])

In [135]:
l = bokeh_ui_utils.get_all_module_centroids(full_filtered_norm_df, leiden_label_df_round_1)

Module 0 is always the largest for leiden and WGCNA; need to check about CLR/MCL

In [136]:
cdf = bokeh_ui_utils.get_module_centroid_df(full_filtered_norm_df, leiden_label_df_round_1)

Index(['TTHERM_ID', 'Ll_GSM283687', 'Ll_GSM284355', 'Ll_GSM284362',
       'Lm_GSM283690', 'Lm_GSM284357', 'Lm_GSM284363', 'Lh_GSM283691',
       'Lh_GSM284360', 'Lh_GSM284364', 'S0_GSM285363', 'S0_GSM285554',
       'S0_GSM285561', 'S3_GSM285542', 'S3_GSM285555', 'S3_GSM285562',
       'S6_GSM285543', 'S6_GSM285556', 'S6_GSM285563', 'S9_GSM285544',
       'S9_GSM285564', 'S15_GSM285559', 'S15_GSM285566', 'S24_GSM285547',
       'S24_GSM285560', 'C0_GSM285570', 'C0_GSM656230', 'C4_GSM285574',
       'C4_GSM285588', 'C4_GSM656234', 'C6_GSM285575', 'C6_GSM656232',
       'C8_GSM285576', 'C8_GSM285590', 'C10_GSM285578', 'C10_GSM285591',
       'C12_GSM285579', 'C12_GSM285592', 'C12_GSM656237', 'C14_GSM285580',
       'C14_GSM285593', 'C14_GSM656238', 'C16_GSM285582', 'C16_GSM285595',
       'C16_GSM656239', 'C18_GSM285583', 'C18_GSM285596', 'C18_GSM656240'],
      dtype='object')
Index(['TTHERM_ID', 'label'], dtype='object')


In [137]:
# arr_df = arrange_modules(full_filtered_norm_df, leiden_label_df_round_1, 'leiden', 'full') # FIXME
arr_df = leiden_label_df_round_1

In [138]:
bokeh_ui_utils.arrange_modules(full_filtered_df, leiden_label_df_round_1, 'full')


Index(['TTHERM_ID', 'Ll_GSM283687', 'Ll_GSM284355', 'Ll_GSM284362',
       'Lm_GSM283690', 'Lm_GSM284357', 'Lm_GSM284363', 'Lh_GSM283691',
       'Lh_GSM284360', 'Lh_GSM284364', 'S0_GSM285363', 'S0_GSM285554',
       'S0_GSM285561', 'S3_GSM285542', 'S3_GSM285555', 'S3_GSM285562',
       'S6_GSM285543', 'S6_GSM285556', 'S6_GSM285563', 'S9_GSM285544',
       'S9_GSM285564', 'S15_GSM285559', 'S15_GSM285566', 'S24_GSM285547',
       'S24_GSM285560', 'C0_GSM285570', 'C0_GSM656230', 'C4_GSM285574',
       'C4_GSM285588', 'C4_GSM656234', 'C6_GSM285575', 'C6_GSM656232',
       'C8_GSM285576', 'C8_GSM285590', 'C10_GSM285578', 'C10_GSM285591',
       'C12_GSM285579', 'C12_GSM285592', 'C12_GSM656237', 'C14_GSM285580',
       'C14_GSM285593', 'C14_GSM656238', 'C16_GSM285582', 'C16_GSM285595',
       'C16_GSM656239', 'C18_GSM285583', 'C18_GSM285596', 'C18_GSM656240'],
      dtype='object')
Index(['TTHERM_ID', 'label'], dtype='object')
19152


,TTHERM_ID,label
730,TTHERM_00216000,1160
779,TTHERM_00535660,1160
820,TTHERM_00537170,1160
1193,TTHERM_00566900,1160
1459,TTHERM_00419710,1160
...,...,...
11648,YF00019924.t1,1183
11815,YF00020165.t1,1183
13965,TTHERM_00384860,1183
16305,TTHERM_01332120,1183


In [139]:
arr_df.loc[arr_df['label'] == 0]

,TTHERM_ID,label
2120,YF00003346.t1,0
2124,YF00003359.t1,0
2254,YF00003639.t1,0
2300,YF00003768.t1,0
2301,YF00003769.t1,0
2447,YF00004038.t1,0
2458,YF00004063.t1,0
5005,YF00008259.t1,0
5027,YF00008319.t1,0
5058,YF00008373.t1,0


In [140]:
arr_df.loc[arr_df['TTHERM_ID'] == 'TTHERM_01386050']

,TTHERM_ID,label
12945,TTHERM_01386050,3


In [141]:
leiden_label_df_round_1.loc[leiden_label_df_round_1['TTHERM_ID'] == 'TTHERM_01386050']

,TTHERM_ID,label
12945,TTHERM_01386050,3


In [142]:
leiden_label_df_round_1_sorted = leiden_label_df_round_1.sort_values(by='label')
leiden_label_df_round_1_sorted.head()

,TTHERM_ID,label
17013,YF00029847.t1,0
9083,YF00015709.t1,0
16202,YF00027629.t1,0
9076,YF00015700.t1,0
2300,YF00003768.t1,0


In [143]:
# for MCL
def leiden_clustering_df_to_csv(df, export_file_path):
    current_module = list(df['label'].values)[0]

    new_cluster = True

    with open(export_file_path, 'w') as f:
        for idx, row in df.iterrows():
            if current_module != row['label']:
                current_module = row['label']
                f.write('\n')
                new_cluster =True

            elif not new_cluster:
                f.write('\t')
            
            f.write(row['TTHERM_ID'])
            new_cluster = False

        f.write('\n')
    
    print(export_file_path, 'successfully exported.')

In [144]:
# leiden_clustering_df_to_csv(leiden_label_df_round_1_sorted, './leiden_clustering.labels')

# for la_param, df in leiden_label_df_cpms.items():
#     leiden_clustering_df_to_csv(df, f'./leiden_clustering_cpm_{str(la_param)}.labels')

In [145]:
# with open('./leiden_clustering.labels', 'r') as f:
#     num_leiden_clusters = f.readlines()
#     print(len(num_leiden_clusters))

# print(max(list(leiden_label_df_round_1_sorted['label'].values)) + 1) # module numbering starts at 0

In [146]:
# rcl_mcl_df.to_csv('./test_nn3_leiden_label_df_round_1_rearranged.csv')

In [147]:
# rcl_mcl_df = pd.read_csv('./mcl_rcl/mcl_rcl_label_df_round_1.csv')
# rcl_mcl_df

In [148]:
# for gene in gene_list_test:
#     print((rcl_mcl_df.loc[rcl_mcl_df['TTHERM_ID'] == gene])['label'].values[0])